Import libraries

In [32]:
#https://github.com/sudhanshusks/twitter_bot/blob/master/bigram_text_generate.ipynb
import json
import numpy as np
import os
import re
import nltk
from nltk.corpus import gutenberg

# Making language model:

In [33]:
nltk.download('punkt')
from nltk.util import ngrams
import string
from nltk import SimpleGoodTuringProbDist
from math import log
import random
from nltk.corpus import twitter_samples
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [34]:
path = '/content/drive/Othercomputers/My Laptop/UNT/Fall 2022/5290 NLP/Python/Bigram_text_generate/'
 # rU means "read", and handles line endings
file= open( path + "sample.txt")
raw= file.read()
print(raw[:11])
tokens= nltk.word_tokenize(raw)
#sample = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "sample.txt")
#test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
print(tokens[:30])
text= nltk.Text(tokens)
print(text[:20])
file.close()

A Twitter b
['A', 'Twitter', 'bot', 'is', 'a', 'type', 'of', 'bot', 'software', 'that', 'controls', 'a', 'Twitter', 'account', 'via', 'the', 'Twitter', 'API', '.', '[', '1', ']', 'The', 'social', 'bot', 'software', 'may', 'autonomously', 'perform', 'actions']
['A', 'Twitter', 'bot', 'is', 'a', 'type', 'of', 'bot', 'software', 'that', 'controls', 'a', 'Twitter', 'account', 'via', 'the', 'Twitter', 'API', '.', '[']


In [35]:
tokens= nltk.corpus.gutenberg.words()
#sample = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "sample.txt")
#test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
print(tokens[:30])
text= nltk.Text(tokens)
print(text[:20])
len(tokens)

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed']
['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich']


2621613

# Unigram Perplexity with add 1 smoothing \ Laplacian smoothing

In [36]:
nltk.download('stopwords')

stopwords= set(nltk.corpus.stopwords.words('english'))
#stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
tokens[-20:]

['knob',
 'really',
 'undoing',
 ',',
 'turning',
 '--',
 'so',
 'now',
 'finally',
 ',',
 'Good',
 '-',
 'bye',
 '--',
 'and',
 'hail',
 '!',
 'my',
 'Fancy',
 '.']

In [38]:
unigram= [w.lower() for w in tokens if w not in stopwords]
#eliminate repeated words
unigram= list(set(unigram))
print(len(unigram))

42334


In [39]:
spl= int(95*len(unigram)/100)
train_corpus= unigram[:spl]
test_corpus= unigram[spl:]


In [44]:
freq_1gram= nltk.FreqDist(train_corpus)
len_train= len(train_corpus)
vocab= len(set(train_corpus))

In [45]:
def unigram_prob_with_add1smoothing(word):
    return (freq_1gram[word]+1)/(len_train+vocab)

In [46]:
def entropy(n, text):
        e = 0.0
        text = ["<s>"] + text + ["</s>"]
        for i in range(n - 1, len(text)):
            context = text[i - n + 1:i]
            token = text[i]
            #print(str(context)+"    "+token)
            e += logprob(token, context)
        return e / float(len(text) - (n - 1))


def logprob(word, context):
    if len(context)==0:
        p=unigram_prob_with_add1smoothing(word)
    elif len(context)==1:
        p=bigram_prob_with_add1smoothing(context[0], word)
    else:
        p=trigram_prob_with_add1smoothing(context[0], context[1], word)
    return -p*log(p , 2)


def perplexity(n, text):
      return pow(2.0, entropy(n, text))

In [47]:
perplexity(1, test_corpus)

1.000140437941288

In [48]:
perplexity(1, train_corpus)

1.0002636496062822

In [49]:

bigram= [a for a in ngrams(unigram, 2, pad_left=True,pad_right=True,left_pad_symbol='<s>', right_pad_symbol="</s>")]
print(bigram[:25])
len(bigram)

[('<s>', 'saukara'), ('saukara', 'despondence'), ('despondence', 'giddalti'), ('giddalti', 'memorandum'), ('memorandum', 'winefat'), ('winefat', 'straine'), ('straine', 'elead'), ('elead', 'digressions'), ('digressions', 'eleusis'), ('eleusis', 'corban'), ('corban', 'italian'), ('italian', 'primarily'), ('primarily', 'unconcerned'), ('unconcerned', 'disadvantages'), ('disadvantages', 'membrane'), ('membrane', 'hjckrrh'), ('hjckrrh', 'unconsciously'), ('unconsciously', 'impiously'), ('impiously', 'hero'), ('hero', 'sage'), ('sage', 'otter'), ('otter', '_lady_'), ('_lady_', 'dozen'), ('dozen', 'exasperate'), ('exasperate', 'seconded')]


42335

In [50]:
cfreq_2gram = nltk.ConditionalFreqDist(bigram)
cprob_2gram = nltk.ConditionalProbDist(cfreq_2gram, nltk.MLEProbDist)
def bigram_prob_with_add1smoothing(word1, word2):
    cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
    return cprob_2gram_add1
#print(cfreq_2gram.tabulate(conditions= ['khalifa', 'romance', 'tourist'], samples=range(100) ))
print(cfreq_2gram['travel'].most_common(20))

[('latencies', 1)]


In [51]:
perplexity(2, test_corpus)

1.0004705404779402

In [52]:
perplexity(2, train_corpus)

1.0004706384307371

In [53]:

def bigram_prob_with_gtsmoothing(word1, word2):
    return SimpleGoodTuringProbDist(word1, word2)

In [54]:
def bigram_prob_with_add1smoothing(word1, word2):
    cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
    return cprob_2gram_add1

In [55]:
per_value_2gram= perplexity(2, test_corpus)
per_value_2gram

1.0004705404779402

In [56]:
trigram= [a for a in ngrams(unigram, 3, pad_left= True, pad_right=True, left_pad_symbol= "<s>", right_pad_symbol="</s>")]
len(trigram)

42336

In [57]:
trigrams_as_bigrams=[]
trigrams_as_bigrams.extend([((t[0],t[1]), t[2]) for t in trigram])
cfreq_3gram = nltk.ConditionalFreqDist(trigrams_as_bigrams)

In [58]:
print(trigrams_as_bigrams[:40])
cfreq_3gram[('lanchar', 'oshua')]

[(('<s>', '<s>'), 'saukara'), (('<s>', 'saukara'), 'despondence'), (('saukara', 'despondence'), 'giddalti'), (('despondence', 'giddalti'), 'memorandum'), (('giddalti', 'memorandum'), 'winefat'), (('memorandum', 'winefat'), 'straine'), (('winefat', 'straine'), 'elead'), (('straine', 'elead'), 'digressions'), (('elead', 'digressions'), 'eleusis'), (('digressions', 'eleusis'), 'corban'), (('eleusis', 'corban'), 'italian'), (('corban', 'italian'), 'primarily'), (('italian', 'primarily'), 'unconcerned'), (('primarily', 'unconcerned'), 'disadvantages'), (('unconcerned', 'disadvantages'), 'membrane'), (('disadvantages', 'membrane'), 'hjckrrh'), (('membrane', 'hjckrrh'), 'unconsciously'), (('hjckrrh', 'unconsciously'), 'impiously'), (('unconsciously', 'impiously'), 'hero'), (('impiously', 'hero'), 'sage'), (('hero', 'sage'), 'otter'), (('sage', 'otter'), '_lady_'), (('otter', '_lady_'), 'dozen'), (('_lady_', 'dozen'), 'exasperate'), (('dozen', 'exasperate'), 'seconded'), (('exasperate', 'secon

FreqDist({})

In [59]:
cfreq_2gram['transpired']

FreqDist({'controllable': 1})

In [60]:
from collections import Counter, OrderedDict
from operator import itemgetter
import random

In [67]:
test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
test_unigram = [w.strip(string.punctuation).lower() for w in test.words()]
test_unigram= [word for word in test_unigram if word not in stopwords]

test_bigram= [a for a in ngrams(test_unigram, 2)]
len(test_bigram)

test_trigram= [a for a in ngrams(test_unigram, 3)]
len(test_trigram)

test_frequency= nltk.FreqDist(test_bigram)
test_sorted_freq= OrderedDict(reversed(sorted(test_frequency.items(), key= itemgetter(1))))

test_cfreq_2gram= nltk.ConditionalFreqDist(test_bigram)

test_cprob_2gram= nltk.ConditionalProbDist(test_cfreq_2gram, nltk.MLEProbDist)

KeyboardInterrupt: ignored